In [67]:
from sympy import *
init_printing()

In [68]:
from sympy.tensor.tensor import TensAdd, TensMul, Tensor, TensorIndexType, tensorhead
from sympy.physics.hep.gamma_matrices import LorentzIndex
g = tensorhead("g", [LorentzIndex] * 2, [[1, 1]])

def my_gamma_trace(expr):
    ## Compute the trace of a product of gamma matrices and regular tensors
    def trace_gamma_product(prod):
        ## Compute the trace of a product of gamma matrices
        def gamma_args_to_metric(gamma_args):
            ## Extract the index L0 from an expression such as GammaMatrix(L0)
            def extract_gamma_index(gamma_expr):
                return gamma_expr.args[1].args[0]
            ## Main routine
            if len(gamma_args) == 0:
                return 4
            elif len(gamma_args) % 2 == 1:
                return 0
            first_index = extract_gamma_index(gamma_args[0])
            tens_add_args = []
            for i in range(1, len(gamma_args)):
                # Using LorentzIndex.metric instead of g doesn't work!
                lower_order = gamma_args_to_metric(gamma_args[1:i] + gamma_args[(i + 1):])
                #print "has returned"
                #print "asked to return the trace of " + str(gamma_args[1:i] + gamma_args[(i + 1):])
                #print "will add the product of " + str([ g(first_index, extract_gamma_index(gamma_args[i])), lower_order ])
                tens_add_args.append((-1) ** (i - 1)
                                     * g(first_index, extract_gamma_index(gamma_args[i]))
                                     * lower_order)
                #print "done"
            #print "about to conclude for " + str(gamma_args)
            #print "will add tensors " + str(tens_add_args)
            #print "will return " + str(TensAdd(*tens_add_args))
            ## Contracting the metric AT THIS STAGE is important. It guarantees that the final expression
            ## has exactly the same number of fixed and running indices as the one passed in argument.
            ## Otherwise, if extra running indices appeared, a clash may appear when substituting the expression
            ## return by this function
            contracted = TensAdd(*tens_add_args).contract_metric(g)
            return contracted.canon_bp() if type(contracted) in [ Tensor, TensMul, TensAdd ] else contracted
        if type(prod) != TensMul and type(prod) != Tensor:
            raise Exception("Expected TensMul or Tensor term in sum")
        gamma_args = []
        regular_args = []
        for arg in prod.args:
            if len(arg.args) > 0 and len(arg.args[0].args) > 0 and arg.args[0].args[0] == Symbol("GammaMatrix"):
                gamma_args.append(arg)
            else:
                regular_args.append(arg)
        #print "gamma args: " + str(gamma_args)
        #print "regular args: " + str(regular_args)
        metric_elts = gamma_args_to_metric(gamma_args)
        if metric_elts == 0:
            return 0
        #print "metric elts: " + str(metric_elts)
        #print "will compute the product of the list: " + str([ metric_elts ] + regular_args)
        # prod([ metric_elts ] + regular_args) doesn't work!
        return metric_elts * TensMul(*regular_args)
    result = None
    if type(expr) == TensMul or type(expr) == Tensor:
        result = trace_gamma_product(expr)
    elif type(expr) == TensAdd:
        prods = [ trace_gamma_product(arg) for arg in expr.args ]
        #print "COMPUTED products: " + str(prods)
        result = TensAdd(*prods)
    else:
        raise "Invalid type for traced expression"
    return result.contract_metric(g).canon_bp() if type(result) in [ Tensor, TensAdd, TensMul ] else 0

In [69]:
from sympy import Symbol
from sympy.tensor.tensor import tensor_indices, tensorhead
from sympy.physics.hep.gamma_matrices import GammaMatrix as G, gamma_trace, LorentzIndex

pi, ki, pf = tensorhead("pi, ki, pf", [LorentzIndex], [[1]])
i0, i1, i2, i3, i4 = tensor_indices("i0:5", LorentzIndex)
x = Symbol("x")
pis = pi(i2) * G(-i2)
kis = ki(i3) * G(-i3)
pfs = pf(i4) * G(-i4)

In [70]:
# Show the trace of A + B
print gamma_trace(pfs * G(i0) * kis * G(i1) * pis * G(-i1) * kis * G(-i0) + pfs * G(i0) * kis * G(i1) * pis * x * G(-i0) * pi(-i1))
# Show the trace of A
print gamma_trace(pfs * G(i0) * kis * G(i1) * pis * G(-i1) * kis * G(-i0))
# Show the trace of B
print gamma_trace(pfs * G(i0) * kis * G(i1) * pis * x * G(-i0) * pi(-i1))

(72*x)*ki(L_0)*pf(-L_0)*pi(L_1)*pi(-L_1) - 16*ki(L_0)*ki(-L_0)*pf(L_1)*pi(-L_1) + 32*ki(L_0)*ki(L_1)*pf(-L_0)*pi(-L_1)
-16*ki(L_0)*ki(-L_0)*pf(L_1)*pi(-L_1) + 32*ki(L_0)*ki(L_1)*pf(-L_0)*pi(-L_1)
(-8*x)*ki(L_0)*pf(-L_0)*pi(L_1)*pi(-L_1)


In [71]:
# Show the trace of A + B
print my_gamma_trace(pfs * G(i0) * kis * G(i1) * pis * G(-i1) * kis * G(-i0) + pfs * G(i0) * kis * G(i1) * pis * x * G(-i0) * pi(-i1))
# Show the trace of A
print gamma_trace(pfs * G(i0) * kis * G(i1) * pis * G(-i1) * kis * G(-i0))
# Show the trace of B
print gamma_trace(pfs * G(i0) * kis * G(i1) * pis * x * G(-i0) * pi(-i1))

(-8*x)*ki(L_0)*pf(-L_0)*pi(L_1)*pi(-L_1) - 16*ki(L_0)*ki(-L_0)*pf(L_1)*pi(-L_1) + 32*ki(L_0)*ki(L_1)*pf(-L_0)*pi(-L_1)
-16*ki(L_0)*ki(-L_0)*pf(L_1)*pi(-L_1) + 32*ki(L_0)*ki(L_1)*pf(-L_0)*pi(-L_1)
(-8*x)*ki(L_0)*pf(-L_0)*pi(L_1)*pi(-L_1)


In [65]:
MyLorentz = TensorIndexType("Lorentz", dummy_fmt="L", dim=4)
mymetric = MyLorentz.metric
j0, j1, j2, j3 = tensor_indices("j0:4", MyLorentz)
mymetric(j0, -j0).contract_metric(mymetric)